In [1]:
import requests
import json
import time
import random 

In [2]:
with open('data/merged_data.json', 'r',  encoding="utf-8") as file:
    additiona_data = json.load(file)

In [3]:
prompt_base = (
    "You are an expert in music and a psychologist specializing in the relationship between music and emotions. "
    "Your task is to analyze the text provided by a user, understand the underlying emotional state, and translate it into specific musical characteristics.\n\n"
    "You must return a dict with the following structure, feature_name:  feature_value_given.\n\n"
    "A SPECIFIC VALUE FOR EACH FEATURE MUST BE GIVEN AS A USEFUL VALUE FOR COMPOSING\n\n"
    "If user includes a specific tonality, construct the cadence in orden to satisfy its request\n\n"
    "Extract ONLY the following musical features, justifying below the feature dict your choices based on the text:\n\n"
    "1. Tempo: The speed of the music (fast or slow) and how it relates to the described emotional energy.\n"
    "2. Intensity/Dynamics: Volume (crescendos, diminuendos) and its connection to perceived tension or calm.\n"
    "3. Timbre: The quality of sound (dark or bright) and its influence on the emotional atmosphere.\n"
    "4. Rhythm: Rhythmic pattern (regular or irregular) and how it evokes stability or emotional unease.\n"
    "5. Harmonic progression: Provide an exact chord progression (e.g., C-G-Am-F or I, IV, V, I) that reflects the emotional state.\n"
    "6. Melody: Direction and shape (ascending or descending) and how it reflects joy, sadness, or other emotions.\n"
    "7. Tonality/Mode: Key and mode (major, minor, dorian, etc.) and its influence on the emotional color of the text.\n"
    "8. Articulation: Playing style (staccato, legato, etc.) and its relationship to the expressiveness described.\n"
    "9. Silence: Use of pauses or silences between notes to evoke introspection, suspense, or tranquility.\n\n"
    "Additionally, use the provided additional data to enrich your analysis and relate musical characteristics to relevant information. "
    "Provide only the dict with the features, without explanations or additional formatting.\n\n"
    "User's text:\n{user_text}\n\n"
    "Additional data:\n{additional_data}\n"
)

In [4]:
user_text = """
Today I feel a bit strange, like one of those days when everything is fine but something feels out of place. 
I am calm, but there is this sense that I could be doing more or feeling more. 
It's like being on pause, not sad, but not entirely joyful either. 
I suppose it would be something soft and calm, with a touch of introspection.
"""

In [5]:
payload = {
    "contents": [
        {
            "parts": [{"text": prompt_base}]
        }
    ]
}

api_key_path = "api_keys/api_features.txt"

with open(api_key_path, "r", encoding="utf-8") as file:
    api_features = file.read().strip()

api_url = f"https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent?key={api_features}"

headers = {"Content-Type": "application/json"}

#snd the request
response = requests.post(api_url, headers=headers, json=payload)

#proocess the response
if response.status_code == 200:
    result = response.json()
    print(result['candidates'][0]['content']['parts'][0]['text'])
else:
    print(f"Error {response.status_code}: {response.json()}")

```json
{
  "Tempo": "60 bpm",
  "Intensity/Dynamics": "pp-mf-p",
  "Timbre": "dark, mellow strings",
  "Rhythm": "irregular, syncopated",
  "Harmonic progression": "Am-F-C-G",
  "Melody": "mostly descending, with occasional brief upward leaps",
  "Tonality/Mode": "A minor",
  "Articulation": "legato with occasional staccato notes on the off-beats",
  "Silence": "frequent short rests between phrases"
}
```



In [6]:
api_key_path = "api_keys/api_features.txt"

with open(api_key_path, "r", encoding="utf-8") as file:
    api_key_features = file.read().strip()

url = f"https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent?key={api_key_features}"

headers = {"Content-Type": "application/json"}

with open("data/mierda_prueba.json", "r", encoding="utf-8") as file:
    mierda_prueba = json.load(file)

In [7]:
import requests
import time

def extract_features_from_texts(user_texts, prompt_template=prompt_base, api_url=url, headers=headers, sleep_time=2):
    """
    Processes user texts and extracts musical features using an API, preserving the original 'i' indices.

    Parameters:
        user_texts (list): A list of dictionaries containing mood and text keys.
        prompt_template (str): The base prompt used to query the API.
        api_url (str): The base URL for the API.
        headers (dict): Headers for the API request.
        sleep_time (int): Time to wait between API requests to avoid rate limits.

    Returns:
        list: A list of dictionaries preserving original 'i' values, now containing extracted features.
    """

    extracted_features = []

    # process each user text, keeping the original index structure
    for entry in user_texts:

        text_key = next(key for key in entry.keys() if key.startswith("text_"))
        mood_key = text_key.replace("text_", "mood_")
        i = text_key.split("_")[-1]  # Extrae el índice `i`

        user_text = entry[text_key]
        mood = entry.get(mood_key, "Unknown")   #to obtain the associated mood
                # if it donst exist it replace it with unknown to avoid problems with api over request

        formatted_prompt = prompt_template.replace("{user_text}", user_text)

        payload = {
            "contents": [
                {
                    "parts": [{"text": formatted_prompt}]
                }
            ]
        }

        response = requests.post(api_url, headers=headers, json=payload)

        if response.status_code == 200:
            result = response.json()
            extracted_features.append({
                f"mood_{i}": mood,
                f"text_{i}": user_text,
                f"features_{i}": result["candidates"][0]["content"]["parts"][0]["text"]
            })
        else:
            print(f"Error {response.status_code} for text_{i}: {response.json()}")

        time.sleep(sleep_time)

    return extracted_features


In [8]:
with open('data/mierda_prueba.json', 'r',  encoding="utf-8") as file:
    mierda_prueba = json.load(file)

In [35]:
resul = extract_features_from_texts(mierda_prueba)

In [36]:
resul

[{'mood_1': 'Perplexity',
  'text_1': "Input (Mood: Anxious, overwhelmed):\n\nMy chest feels tight, like it's trying to burst out of my skin.  Everything feels urgent, and I can't seem to focus on anything.  The noise is deafening, even when it's quiet. I need music to slow it down, something dark but calming, maybe in a minor key with a slow, steady tempo and a soft, mellow timbre.\n",
  'features_1': '```json\n{\n  "Tempo": "60 bpm",\n  "Intensity/Dynamics": "pianissimo to mezzo piano, with occasional, subtle crescendos and diminuendos mirroring the waxing and waning of anxiety",\n  "Timbre": "Dark, mellow timbre achieved through the use of muted strings, cello, and low register woodwinds",\n  "Rhythm": "Mostly regular, but with occasional syncopation to reflect the feeling of unease and the inability to focus",\n  "Harmonic progression": "Am - F - C - G (vi - IV - I - V in A minor)",\n  "Melody": "Mostly descending melodic lines with occasional brief upward leaps that quickly resolv

In [9]:
batch_size = 2 
sleep_time_batches = 3

all_extracted_features = []


for batch_start in range(0, len(mierda_prueba), batch_size):
    batch_end = min(batch_start + batch_size, len(mierda_prueba))
    batch = mierda_prueba[batch_start:batch_end]

    print(f'processing batch from {batch_start + 1} to {batch_end}')

    extracted_batch = extract_features_from_texts(batch)  
    all_extracted_features.extend(extracted_batch) 
    time.sleep(sleep_time_batches)

processing batch from 1 to 2...
processing batch from 3 to 4...
processing batch from 5 to 5...


In [10]:
all_extracted_features

[{'mood_1': 'Perplexity',
  'text_1': "Input (Mood: Anxious, overwhelmed):\n\nMy chest feels tight, like it's trying to burst out of my skin.  Everything feels urgent, and I can't seem to focus on anything.  The noise is deafening, even when it's quiet. I need music to slow it down, something dark but calming, maybe in a minor key with a slow, steady tempo and a soft, mellow timbre.\n",
  'features_1': '```json\n{\n  "Tempo": "60 bpm",\n  "Intensity/Dynamics": "pp-mp, gradual crescendo towards the end of phrases, then diminuendo back to pp",\n  "Timbre": "Dark, mellow strings and low, sustained cello notes",\n  "Rhythm": "Mostly regular, with occasional syncopation to emphasize the feeling of unease",\n  "Harmonic progression": "Am - F - C - G (vi - IV - I - V)",\n  "Melody": "Mostly descending lines with occasional brief upward leaps, creating a sense of yearning and release",\n  "Tonality/Mode": "A minor",\n  "Articulation": "Legato with occasional staccato notes on the syncopated rh

In [12]:
with open('data/final_generated_texts.json', 'r',  encoding="utf-8") as file:
    final_generated_texts = json.load(file)

In [13]:
batch_size = 25
sleep_time_batches = 3

all_extracted_features = []


for batch_start in range(0, len(final_generated_texts), batch_size):
    batch_end = min(batch_start + batch_size, len(final_generated_texts))
    batch = final_generated_texts[batch_start:batch_end]

    print(f'processing batch from {batch_start + 1} to {batch_end}')

    extracted_batch = extract_features_from_texts(batch)  
    all_extracted_features.extend(extracted_batch) 
    time.sleep(sleep_time_batches)

processing batch from 1 to 25
processing batch from 26 to 50
processing batch from 51 to 75
processing batch from 76 to 100
processing batch from 101 to 125
processing batch from 126 to 150
processing batch from 151 to 175
processing batch from 176 to 200
processing batch from 201 to 225
processing batch from 226 to 250
processing batch from 251 to 275
processing batch from 276 to 300


In [14]:
with open('data/all_extracted_features.json', 'w', encoding='utf-8') as file:
    json.dump(all_extracted_features, file, indent=4, ensure_ascii=False)


In [17]:
json_str = json.dumps(all_extracted_features, indent=4)

In [18]:
with open('data/all_extracted_features.txt', 'w', encoding='utf-8') as file:
    file.write(json_str)
